In [1]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.common.exceptions import TimeoutException

In [2]:
chrome_path = 'C:/Users/Redata/Downloads/read/chromedriver_win32/chromedriver.exe'

# open the our selenium tools
driver = webdriver.Chrome(chrome_path)

In [3]:
data = pd.DataFrame(columns=['link', 'penilaian', 'price', "spek", 
                             "seller_name", "seller_rate"])

### Gathering the link of product 
**We limit up to page-4 which equal to 200 link product.**

In [4]:
link = []
#loop the page
for page in range(4):
    
    keyword = "case hp"
    
    keyword1 = keyword.split()[0]
    keyword2 = keyword.split()[1]
    
    #the main link strcuture based on keyword and the product filter
    main_link = f"https://shopee.co.id/search?brands=14%2C15%2C350&freeShipping=true&keyword={keyword1}%20{keyword2}&labelIds=1000006%2C1000246&locations=Jabodetabek&maxPrice=200000&noCorrection=true&page={page}&ratingFilter=4"
    
    driver.get(main_link)
    for i in range(10):
        sc = np.random.rand()
        driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*{sc});")
    
    new_link = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
    
    while len(new_link) != 50:
            driver.implicitly_wait(1)
            new_link = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
            
    #get the html data which contains link list
    new_link = driver.find_elements_by_xpath('//a[@data-sqe="link"]')
    
    #extract the link and appending to 'link' list 
    for links in new_link:
        p_link = links.get_attribute('href')
        link.append(p_link)

In [5]:
# make empty dataframe 
data = pd.DataFrame(columns=['link', 'penilaian', 'price', "spek", 
                             "seller_name", "seller_rate"])

### Gathering the product  and supplier description based on link
**Looping for each of the 200 product links that we got earlier.**

In [6]:
for links in link:
    driver.get(links)
    driver.implicitly_wait(2)
    
    #scroll down the page to make sure it contains the information we need
    driver.execute_script(f"window.scrollTo(0, document.body.scrollHeight*0.7);")
    driver.implicitly_wait(2)
    
    # gathering the html data (our raw data)
    html = driver.page_source
    soup = bs(html, "lxml")
    seller = soup.find("div", class_ = "_1zBnTu page-product__shop")
        
    while seller == None:
        driver.implicitly_wait(1)
        html = driver.page_source
        soup = bs(html, "lxml")
        seller = soup.find("div", class_ = "_1zBnTu page-product__shop")
            
    penilaian = []
    #find the review infomation
    nilai = soup.find(class_ = "flex _32fuIU")
    
    ra = nilai.findAll(class_ = re.compile('^_3Oj5_n'))
    
    penilaian.append(ra[0].text)
    penilaian.append(ra[1].text)
    penilaian.append(nilai.find(class_ = "_22sp0A").text)
    
    #find the price of product
    price = soup.find(class_ = "_3n5NQx").text
    
    #find the product spesification
    spek = soup.find_all(class_ = "kIo6pj")
    spesifikasi = []
    for s in spek:
        try:
            val = s.text
        except:
            val = np.nan
                
        spesifikasi.append(val)
    
    
    #extract the seller information
    seller = soup.find("div", class_ = "_1zBnTu page-product__shop")
    seller_rate = []
    try:
        name = seller.find(class_ = "_3Lybjn").text
        seller_ratess = seller.findAll(class_ = re.compile('^_1rsHot'))
        for ra in seller_ratess:
            seller_rate.append(ra.text)
    except:
        name = np.nan
        seller_rate.append(np.nan)
            
    #append them to our empty dataframe
    data = data.append({'link':links, 'penilaian':penilaian, 'price':price, 'spek':spesifikasi,
                        "seller_name":name, "seller_rate":seller_rate}, ignore_index=True)

In [7]:
data.sample(10)

,link,penilaian,price,spek,seller_name,seller_rate
82,https://shopee.co.id/Samsung-Neon-Flip-Cover-G...,"[5.0, 15, 19]",Rp49.000,[KategoriShopeeHandphone & AksesorisCasing & C...,felixindo,"[11,5RB, 1,3RB, 57%, hitungan jam, 3 tahun lal..."
50,https://shopee.co.id/KESSING-CASSING-HOUSSING-...,"[4.7, 15, 24]",Rp65.000,[KategoriShopeeHandphone & AksesorisSpare Part...,amiencell40,"[2,2RB, 1,7RB, 57%, hitungan jam, 29 bulan lal..."
96,https://shopee.co.id/CASING-HOUSING-SAMSUNG-FU...,"[4.2, 5, 5]",Rp65.000,[KategoriShopeeHandphone & AksesorisCasing & C...,smileparts,"[490, 1,2RB, 57%, hitungan jam, 36 bulan lalu,..."
40,https://shopee.co.id/SAMSUNG-J1-J2-PRIME-J3-CU...,"[5.0, 91, 1,9RB]",Rp2.950 - Rp14.500,[KategoriShopeeHandphone & AksesorisCasing & C...,jayaraya.id,"[2,7RB, 684, 57%, hitungan jam, 20 bulan lalu,..."
162,https://shopee.co.id/Anti-Crack-Samsung-S9-Plu...,"[5.0, 1, 1]",Rp9.800,[KategoriShopeeHandphone & AksesorisCasing & C...,mandiri888acc,"[1,2RB, 1,6RB, 57%, hitungan jam, 3 tahun lalu..."
67,https://shopee.co.id/CASE-Casing-COVER-ORIGINA...,"[5.0, 3, 3]",Rp96.000,[KategoriShopeeHandphone & AksesorisCasing & C...,jackie_shop,"[2,9RB, 110, 57%, hitungan jam, 4 tahun lalu, ..."
154,https://shopee.co.id/BACKDOOR-BACKCASE-TUTUP-B...,"[4.8, 4, 6]",Rp55.000,[KategoriShopeeHandphone & AksesorisCasing & C...,smileparts,"[490, 1,2RB, 57%, hitungan jam, 36 bulan lalu,..."
0,https://shopee.co.id/CUSTOM-CASE-SOFTCASE-SEMU...,"[4.7, 604, 10RB+]",Rp2.950 - Rp14.500,[KategoriShopeeHandphone & AksesorisCasing & C...,jayaraya.id,"[2,7RB, 684, 57%, hitungan jam, 20 bulan lalu,..."
100,https://shopee.co.id/CASING-CASE-SAMSUNG-M10-M...,"[4.5, 2, 4]",Rp7.500,[KategoriShopeeHandphone & AksesorisCasing & C...,duniacase26,"[268, 485, 57%, hitungan jam, 34 hari lalu, 167]"
178,https://shopee.co.id/SAMSUNG-A50-SILIKON-TPU-L...,"[4.6, 5, 12]",Rp25.000,[KategoriShopeeHandphone & AksesorisCasing & C...,loopee,"[909, 1,9RB, 57%, hitungan jam, 3 tahun lalu, ..."


In [8]:
print ("Number of available supplier equal to:" , data["seller_name"].nunique())

Number of available supplier equal to: 76


### Cleansing the dataset
More specifically seperate the list

In [10]:
# product information
data["star_rating"] = data["penilaian"].str[0]
data["cunt_rating"] = data["penilaian"].str[1]
data["sales"] = data["penilaian"].str[2]

data["category"] = data["spek"].str[0]
data["Merk"] = data["spek"].str[1]
data["Stock"] = data["spek"].str[2]

# supplier information
data["Supplier_Place"] = data["spek"].str[3]
data["total_penilaian"] = data["seller_rate"].str[0]
data["total_product"] = data["seller_rate"].str[1]
data["percentage_reply_m"] = data["seller_rate"].str[2]
data["reply_time_parameter"] = data["seller_rate"].str[3]
data["registered_date"] = data["seller_rate"].str[4]
data["follower"] = data["seller_rate"].str[5]

In [11]:
#save the data
data.to_csv("shopee_scraping_result.csv", index=False)

In [15]:
data.drop(columns=["penilaian", "seller_rate", "spek"]).sample(10)

,link,price,seller_name,star_rating,cunt_rating,sales,category,Merk,Stock,Supplier_Place,total_penilaian,total_product,percentage_reply_m,reply_time_parameter,registered_date,follower
113,https://shopee.co.id/Casing-Housing-Fullset-Sa...,Rp35.000,miaumiaushop,4.6,8,17,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSamsung,BahanPlastik,Model HPj2,"7,7RB",521,57%,hitungan jam,32 bulan lalu,3RB
186,https://shopee.co.id/Case-Anti-Crack-Samsung-A...,Rp12.500,pandawaacc,5.0,1,3,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSamsung,BahanSilikon,Model HPA5 2017,"3,7RB",820,57%,hitungan jam,3 tahun lalu,953
138,https://shopee.co.id/-COD-%E2%9C%94-Xiaomi-Not...,Rp47.500,lapaklama,4.5,2,5,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekXiaomi,Bahan-,Model HPXiaomi Redmi Note 5A Prime,"150,9RB","1,6RB",57%,hitungan jam,4 tahun lalu,"71,9RB"
84,https://shopee.co.id/housing-casing-samsung-J5...,Rp25.000,tahtacom.,4.7,6,7,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSamsung,Bahanplastik,Model HPsamsung j5 j500,178,936,57%,hitungan jam,22 bulan lalu,54
42,https://shopee.co.id/Clear-View-Standing-Cover...,Rp25.000,sauqilahshopdistribution,5.0,2,5,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSamsung,BahanPlastik,Model HPsamsung galaxy s9 dan s9 plus,"3,3RB",93,57%,hitungan jam,27 bulan lalu,1RB
53,https://shopee.co.id/XIAOMI-REDMI-NOTE-5-REDMI...,Rp188.000,joriginal,5.0,2,2,KategoriShopeeHandphone & AksesorisCasing & Co...,BahanHardcase,Model HPXiaomi Note 5 Note 5 Pro,Stok147,"4,7RB","1,9RB",57%,hitungan jam,4 tahun lalu,929
43,https://shopee.co.id/CASING-HOUSING-SAMSUNG-S5...,Rp30.000,acc_ku,4.6,8,10,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSamsung,BahanPlastik,Model HPs5830,"23,4RB",2RB,57%,hitungan jam,3 tahun lalu,"4,1RB"
174,https://shopee.co.id/Case-360-xiaomi-redmi-not...,Rp10.000,nanang1001,5.0,1,1,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekXiaomi,BahanPlastik,Model HPxiaomi redmi note 4x snapdragon,73,146,57%,hitungan jam,32 bulan lalu,69
23,https://shopee.co.id/Soft-Case-Silicon-Xiaomi-...,Rp44.000,smartg3n,5.0,1,2,KategoriShopeeHandphone & AksesorisCasing & Co...,BahanSilikon,Model HPxiaomi redmi S2,Stok20,466,67,57%,hitungan jam,36 bulan lalu,125
172,https://shopee.co.id/Casing-Kesing-Samsung-Gal...,Rp125.000,afrodit,5.0,3,3,KategoriShopeeHandphone & AksesorisCasing & Co...,MerekSAMSUNG,Stok2,Dikirim DariKOTA JAKARTA BARAT - GROGOL PETAMB...,868,532,57%,hitungan jam,4 tahun lalu,343
